# PepperOpenDayDemo

## Environment Setup

In [1]:
# built-in imports
!pip install requests
!pip install speechrecognition==3.8.1
!pip install paramiko
!pip install pandas
!pip install beautifulsoup4
!pip install scipy

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer main

In [1]:
### Naoqi Related Imports ###
import naoqi
from naoqi import qi
from naoqi import ALProxy

### System Related Imports ###
import fileinput, os, shutil, sys, random, threading, time

### Flask Related Imports ###
import json, requests

### Speech Recognition Imports ###
import pandas as pd
import paramiko
import speech_recognition as sr

/usr/local/lib/python2.7/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography.hazmat.backends import default_backend


In [2]:
from prompts import *
import prompts
from fileTransfer import *
import fileTransfer as ft
from displayGeneration import *
import displayGeneration as dg
from interactiveControls import *
import interactiveControls as ic
from idle import *
import idle as idle
from humanInteraction import *
import humanInteraction as hi


# ### Custom File Handing Imports ###
# from fileTransfer import *

# ### Custom Idle Behaviour Imports ###
# from idle import *

# ### Custom Page Display Imports ###
# from displayGeneration import *

# ### Custom Interaction, Behaviour and Display Imports ###
# from interactiveControls import *

# ### Custom Hard-Coded Prompt Imports ###
# from prompts import *

# ### Main Interaction Imports ###
# from humanInteraction import *

## Constants

In [3]:
### Networking Constants ###
GPT_HOST = "10.104.23.208"
PEPPER_HOST = "10.104.23.185"
GPT_PORT = 8891
PEPPER_PORT = 9559

### Audio Processing Constants ###
PEPPER_RECORDINGS_PATH = "/home/nao/microphones/recording.wav"
SERVER_RECORDINGS_PATH = "recordings/recording.wav"

### Display Generation Constants ###
FILE_NAME_TEMP = "display.html"
PEPPER_HTML_PATH = "/home/nao/.local/share/PackageManager/apps/robot-page/html/"  # page.html"
PEPPER_PAGE_LANDING = PEPPER_HTML_PATH + "page.html" # '/home/nao/.local/share/PackageManager/apps/robot-page/html/page.html'
PEPPER_QR_LANDING = PEPPER_HTML_PATH + "webfiles/qr.png"  # switched slash direction
PEPPER_IMG_LANDING = PEPPER_HTML_PATH + "webfiles/img.png"  # switched slash direction
TEXT_BY_ID_PATH = "textbyID.csv"

## MAIN

In [4]:
global idle
global responsesPipeline
global eyes

idle = idling()
eyes = EyesController()
eyes.startEyes("neutral")

try:
    track_head()
    repeat = True

    while True:
        # Empty sentences list
        responsesPipeline = []
        
        # Idle behaviors to keep peppers temperature low
        idle.start_idle_behavior()

        # trigger smart listen
        detect()
        idle.stop()
        
        ## If speech detected record up to 15 seconds of audio
        query = listen()
        # queryCourseCodes returns "repeat" = true if the query search is not successful
        repeat = queryCourseCodes(query, responsesPipeline, eyes)
        
        # get course codes relevant to this
        # if query search is not successful, try the query search again until it is successful
        while repeat:
            query = listen()
            repeat = queryCourseCodes(query, responsesPipeline, eyes)
            
        #ask if they need specific details about a course
        repeat = True
        rcount = 0
        while repeat:
            query = listen()
            repeat = querySpecificCourse(query, responsesPipeline, eyes, rcount)
            rcount += 1
        
except Exception as e:
    print(e)
finally:
    stopListening()
    return_to_default_pos()
    idle = idling() 
    idle.start_idle_behavior()
    print('Goodbye')

copyfile successful
Sent to Pepper
0.335101246834
Detect complete
Recording is starting...
copyfile successful
Sent to Pepper
Timer is reached
Recording is stopping...
Sent from Pepper
array([   0,    0,   -2, ..., -459, -398, -193], dtype=int16) is not JSON serializable
Goodbye


# Yash ka shit

In [ ]:
bumperDetector = ALProxy("ALTouch", constants.PEPPER_HOST, constants.PEPPER_PORT)

memory = ALProxy("ALMemory", constants.PEPPER_HOST, constants.PEPPER_PORT)

# bumperDetector.subscribe("Device/SubDeviceList/Platform/Back/Bumper/Sensor/Value")



touch = bumperDetector.subscriber("TouchChanged")

status = memory.getData("SoundDetected")

print(touch)

print(status)



self.touch.signal.connect(functools.partial(self.onTouched, "TouchChanged"))



def onTouched():

    print("touched")

NameError: name 'ALProxy' is not defined

In [ ]:
bumperDetector = ALProxy("ALTouch", PEPPER_HOST, PEPPER_PORT)

In [ ]:
touch = bumperDetector.session.subscribe("TouchChanged")

AttributeError: 'function' object has no attribute 'subscribe'

In [ ]:
memory = ALProxy("ALMemory", PEPPER_HOST, PEPPER_PORT)

In [ ]:
bumperDetector.session().service

AttributeError: Boost.Python function __module__ unknown.

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/local/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "interactiveControls.py", line 66, in colourEyes
    self.leds.fadeRGB("FaceLeds", 'white', 1)
  File "/pynaoqi-python2.7-2.5.7.1-linux64/lib/python2.7/site-packages/naoqi.py", line 194, in __call__
    return self.__wrapped__.method_missing(self.__method__, *args, **kwargs)
  File "/pynaoqi-python2.7-2.5.7.1-linux64/lib/python2.7/site-packages/naoqi.py", line 264, in method_missing
    raise e
RuntimeError: 	ALLeds::fadeRGB
	Session closed



In [ ]:
dir(bumperDetector.session())

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'async',
 'call',
 'callModule',
 'close',
 'connect',
 'connected',
 'disconnected',
 'endpoints',
 'isConnected',
 'listen',
 'listenStandalone',
 'loadServiceRename',
 'metaObject',
 'registerService',
 'service',
 'serviceRegistered',
 'serviceUnregistered',
 'services',
 'unregisterService',
 'url',
 'waitForService']

In [ ]:
touch = memory.subscribeToEvent()

RuntimeError: 	ALMemory::subscribeToEvent
	Arguments types did not match for subscribeToEvent (resolved to '()'):
  Candidates:
  subscribeToEvent::(sss) (1)
  subscribeToEvent::(ssss) (1)


In [ ]:
dir(bumperDetector.session)

['__call__',
 '__class__',
 '__cmp__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__func__',
 '__get__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'im_class',
 'im_func',
 'im_self']